In [8]:
import os
import json
import shutil
from mintpy.bulk_plate_motion import build_PMM

# Read all the paths from the json file

The velocity files, DEM files, and the geoemtry files are all stored on `marmot-nobak`. These paths are stored in `datasets.json`.

We will:

1. Read the paths from the json file into a dictioanry for future use.
2. Make a backup of the json file

In [3]:
## Build a dictionary to save all the LOSgeom class obj from different datasets (tracks)
all_class = dict()


## Read pre-defined paths
my_json = '../inputs/datasets.json'
with open(my_json) as file:
    pathDict = json.load(file)
    
    
## Save pre-defined paths to a back-up
with open('../inputs/datasets_bak.json', 'w') as file:
    json_string = json.dumps(pathDict, default=lambda o: o.__dict__, sort_keys=True, indent=2)
    file.write(json_string)
    
    
## Read pre-defined paths
my_json = '../inputs/brief.json'
with open(my_json) as file:
    brief = json.load(file)    

In [3]:
if False:

    ## Save new copy files from remote places~
    brief = dict()

    for name in pathDict:
        if 'dem' not in pathDict[name]:
            continue

        brief[name] = dict()

        n, t = name.split()
        outdir = '../in_h5/{}_{}'.format(n,t)
        demf = '{}.dem'.format(outdir)
        geof = '{}_Geo.h5'.format(outdir)
        mskf = '{}_msk.h5'.format(outdir)
        v1f  = '{}_velRaw.h5'.format(outdir)
        v2f  = '{}_vel.h5'.format(outdir)
        v3f  = '{}_velIon.h5'.format(outdir)

        datainfo = pathDict[name]
        fns = []
        fns.append([datainfo['dem'], demf])
        brief[name]['dem'] = os.path.abspath(demf)

        for i, (k, out) in enumerate(zip(['geo','mask','velo1','velo2','veloIon'],[geof,mskf,v1f,v2f,v3f])):
            if not datainfo[k]:
                continue
            home = datainfo['home']
            fns.append([os.path.expanduser(home+'/'+datainfo[k]), out])
            brief[name][k] = os.path.abspath(out)

        for fn in fns:
            print('{}   -->   {}'.format(fn[0], fn[1]))
            shutil.copyfile(fn[0], fn[1])


    ## Save new copy paths to a brief json
    with open('../inputs/brief.json', 'w') as file:
        json_string = json.dumps(brief, default=lambda o: o.__dict__, sort_keys=True, indent=2)
        file.write(json_string)        

In [10]:
# Add Euler pole info into the dictionary

areas = ['Aqaba', 'Aust', 'Makran']
poles = [[1.154, -0.136, 1.444], [1.510, 1.182, 1.215], [-0.085, -0.531, 0.770]]

for name in brief:
    for i, (key, pole) in enumerate(zip(areas, poles)):
        if name.startswith(key):
            print(name, key, pole)
            brief[name]['EP_cart'] = pole

with open('../inputs/brief.json', 'w') as file:
    json_string = json.dumps(brief, default=lambda o: o.__dict__, sort_keys=True, indent=2)
    file.write(json_string)  

Aqaba a087 Aqaba [1.154, -0.136, 1.444]
Aqaba d021 Aqaba [1.154, -0.136, 1.444]
Aqaba d021_bdg Aqaba [1.154, -0.136, 1.444]
Australia d046 Aust [1.51, 1.182, 1.215]
Australia d046_exc Aust [1.51, 1.182, 1.215]
Australia d046_noERA5 Aust [1.51, 1.182, 1.215]
Australia d046_noESD Aust [1.51, 1.182, 1.215]
Australia d119 Aust [1.51, 1.182, 1.215]
Makran a013 Makran [-0.085, -0.531, 0.77]
Makran a086 Makran [-0.085, -0.531, 0.77]
Makran a115 Makran [-0.085, -0.531, 0.77]
Makran a159 Makran [-0.085, -0.531, 0.77]
Makran d020 Makran [-0.085, -0.531, 0.77]


In [15]:
areas = ['Aqaba', 'Aust', 'Makran']
poles = [[1.154, -0.136, 1.444], [1.510, 1.182, 1.215], [-0.085, -0.531, 0.770]]

for i, (area, pole) in enumerate(zip(areas, poles)):
    print('\n\n< {} >'.format(area))
    build_PMM(omega_cart=pole)



< Aqaba >
your input: omega_cartesian; [wx, wy, wz] (mas/yr)
--- Euler Pole and rotation vector ---

Spherical representation:
 Latitude:      51.1764 deg
 Longitude:     353.2786 deg
 Rotation rate: 0.5149 deg / Ma  


Cartesian representation:
 wx:            1.1540 mas / yr
 wy:            -0.1360 mas / yr
 wz:            1.4440 mas / yr
--------------------------------------


< Aust >
your input: omega_cartesian; [wx, wy, wz] (mas/yr)
--- Euler Pole and rotation vector ---

Spherical representation:
 Latitude:      32.3584 deg
 Longitude:     38.0532 deg
 Rotation rate: 0.6306 deg / Ma  


Cartesian representation:
 wx:            1.5100 mas / yr
 wy:            1.1820 mas / yr
 wz:            1.2150 mas / yr
--------------------------------------


< Makran >
your input: omega_cartesian; [wx, wy, wz] (mas/yr)
--- Euler Pole and rotation vector ---

Spherical representation:
 Latitude:      55.0699 deg
 Longitude:     260.9055 deg
 Rotation rate: 0.2609 deg / Ma  


Cartesian re

In [30]:
import pyproj

geod = pyproj.Geod(ellps='WGS84') # Use WGS 1984 ellipsoid.

g1 = geod.inv(36,30,353.27,51.17)
g2 = geod.inv(125,-26,38.05,32.35)
g3 = geod.inv(61,29,260.90,55.06)

print('The region pointing to the ITRF14 PMM Euler pole [km]')
print('\n< Aqaba >')
print('{:.1f} km     azimuth {:.1f} deg'.format(g1[-1]/1e3, g1[1]))
print('\n< Australia >')
print('{:.1f} km     azimuth {:.1f} deg'.format(g2[-1]/1e3, g2[1]))
print('\n< Makran >')
print('{:.1f} km     azimuth {:.1f} deg'.format(g3[-1]/1e3, g3[1]))


The region pointing to the ITRF14 PMM Euler pole [km]

< Aqaba >
4227.2 km     azimuth 107.1 deg

< Australia >
11245.3 km     azimuth 113.6 deg

< Makran >
10499.1 km     azimuth 17.3 deg


In [5]:
from mintpy import bulk_plate_motion

for name in brief:
    base    = os.path.basename(brief[name]['dem']).split('.')[0].split('/')[-1]
    outdir  = os.path.abspath('../in_h5/{}'.format(base))
    mout    = '{}_vel_bmModelRaw.h5'.format(outdir)
    vfile   = brief[name]['velo2']
    gfile   = brief[name]['geo']
    ep_cart = brief[name]['EP_cart']
    ep_str  = [str(ep) for ep in ep_cart]
    iargs   = ['-g', gfile, '-v', vfile, '--om_cart', ep_str[0], ep_str[1], ep_str[2]]
    print('========================================================')
    print(' '.join(iargs)) 
    bulk_plate_motion.main(iargs)
    
    brief[name]['bmModelRaw']  = '{}_vel_bmModelRaw.h5'.format(outdir)
    brief[name]['bmModel']     = '{}_vel_bmModel.h5'.format(outdir)
    brief[name]['velo_bmCorr'] = '{}_vel_bmCorr.h5'.format(outdir)


with open('../inputs/brief.json', 'w') as file:
    json_string = json.dumps(brief, default=lambda o: o.__dict__, sort_keys=True, indent=2)
    file.write(json_string)

-g /net/kraken/nobak/ykliu/2022-BulkMotion/in_h5/Aqaba_a087_Geo.h5 -v /net/kraken/nobak/ykliu/2022-BulkMotion/in_h5/Aqaba_a087_vel.h5 --om_cart 1.154 -0.136 1.444

-----------------------------------
Evaluate bulk motion
-----------------------------------

prepare LOS geometry in geo-coordinates from file: /net/kraken/nobak/ykliu/2022-BulkMotion/in_h5/Aqaba_a087_Geo.h5
read incidenceAngle from file: /net/kraken/nobak/ykliu/2022-BulkMotion/in_h5/Aqaba_a087_Geo.h5
read azimuthAngle   from file: /net/kraken/nobak/ykliu/2022-BulkMotion/in_h5/Aqaba_a087_Geo.h5
convert azimuth angle to heading angle

Form a bulk motion model velocity field
estimate bulk motion by building a plate motion model (e.g., ITRF2014 MORVEL)
your input: omega_cartesian; [wx, wy, wz] (mas/yr)
--- Euler Pole and rotation vector ---

Spherical representation:
 Latitude:      51.1764 deg
 Longitude:     353.2786 deg
 Rotation rate: 0.5149 deg / Ma  


Cartesian representation:
 wx:            1.1540 mas / yr
 wy:       